This project presents an analysis of a retail sales database, focusing on customer profitability, inventory management, and customer purchase trends. Using SQL within a Jupyter Notebook environment, I examined key metrics such as the number of customers, products, and transactions, as well as the distribution of profits among top clients. The analysis identifies high-value customers, highlights products at risk of stockouts, and evaluates customer lifetime value (LTV), which represents the average amount of money a customer generates.

In [11]:
!wget -q -O stores.db https://dq-content.s3.amazonaws.com/600/stores.db
%load_ext sql
%sql sqlite:///stores.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
%%sql
SELECT 'Customers' AS table_name, 13 AS number_of_attributes, COUNT(*) AS number_of_rows 
  FROM Customers
 UNION ALL
 
SELECT 'Products' AS table_name, 9 AS number_of_attributes, COUNT(*) AS number_of_rows 
  FROM Products
 UNION ALL
 
SELECT 'ProductLines' AS table_name, 4 AS number_of_attributes, COUNT(*) AS number_of_rows 
  FROM ProductLines
 UNION ALL
 
SELECT 'Orders' AS table_name, 7 AS number_of_attributes, COUNT(*) AS number_of_rows 
  FROM Orders
 UNION ALL
 
SELECT 'OrderDetails' AS table_name, 5 AS number_of_attributes, COUNT(*) AS number_of_rows 
  FROM OrderDetails
 UNION ALL
 
SELECT 'Payments' AS table_name, 4 AS number_of_attributes, COUNT(*) AS number_of_rows 
  FROM Payments
 UNION ALL
 
SELECT 'Employees' AS table_name, 8 AS number_of_attributes, COUNT(*) AS number_of_rows 
  FROM Employees
 UNION ALL
 
SELECT 'Offices' AS table_name, 9 AS number_of_attributes, COUNT(*) AS number_of_rows 
  FROM Offices

 * sqlite:///stores.db
Done.


table_name,number_of_attributes,number_of_rows
Customers,13,122
Products,9,110
ProductLines,4,7
Orders,7,326
OrderDetails,5,2996
Payments,4,273
Employees,8,23
Offices,9,7


The dataset from the stores.db database provides an overview of a retail business, including customers, products, orders, payments, employees, and offices. The database consists of 8 main tables, with the largest being OrderDetails (2,996 rows) and Orders (326 rows), indicating a substantial volume of transactional data. The Customers table contains 122 entries, while Products lists 110 unique items, reflecting a diverse product portfolio and customer base.

## low stock

In [3]:
%%sql

SELECT productCode, ROUND(SUM(quantityOrdered) * 1.0 / (SELECT quantityInStock
                                                       FROM products
                                                       WHERE orderdetails.productCode = products.productCode), 2) AS low_stock
FROM orderdetails
GROUP BY productCode
ORDER BY low_stock
LIMIT 10;

 * sqlite:///stores.db
Done.


productCode,low_stock
S18_1984,0.09
S24_3432,0.09
S12_2823,0.1
S12_3380,0.1
S18_1589,0.1
S18_2325,0.1
S18_2870,0.1
S18_3482,0.1
S32_2206,0.1
S700_2466,0.1


Low Stock Products: Several products, such as S18_1984 and S24_3432, have a low stock ratio (quantityOrdered / quantityInStock) (as low as 0.09), highlighting the need for timely restocking and potential challanges with inventory management.

## product performance

In [13]:
%%sql
SELECT productCode, 
       SUM(quantityOrdered * priceEach) AS prod_perf
  FROM orderdetails od
 GROUP BY productCode 
 ORDER BY prod_perf DESC
 LIMIT 10;

 * sqlite:///stores.db
Done.


productCode,prod_perf
S18_3232,276839.98
S12_1108,190755.86000000002
S10_1949,190017.96
S10_4698,170686.0
S12_1099,161531.48
S12_3891,152543.02
S18_1662,144959.91
S18_2238,142530.63
S18_1749,140535.6
S12_2823,135767.03


The top-performing products generate significantly higher revenue than others. Focusing marketing efforts and stock availability on these bestsellers can further boost sales and profitability. 

## product line performance

In [23]:
%%sql

WITH low_stock AS (
SELECT productCode, ROUND(SUM(quantityOrdered) * 1.0 / (SELECT quantityInStock
                                                       FROM products
                                                       WHERE orderdetails.productCode = products.productCode), 2) AS low_stock
FROM orderdetails
GROUP BY productCode
ORDER BY low_stock
LIMIT 10)

SELECT products.productName, products.productCode, products.productLine, products.productCode, SUM(orderdetails.quantityOrdered * orderdetails.priceEach) AS product_performence
FROM orderdetails
JOIN products
ON products.productCode = orderdetails.productCode
WHERE orderdetails.productCode IN (SELECT productCode FROM low_stock)

GROUP BY orderdetails.productCode
ORDER BY product_performence DESC;

 * sqlite:///stores.db
Done.


productName,productCode,productLine,productCode_1,product_performence
2002 Suzuki XREO,S12_2823,Motorcycles,S12_2823,135767.03
1976 Ford Gran Torino,S18_3482,Classic Cars,S18_3482,121890.59999999999
1995 Honda Civic,S18_1984,Classic Cars,S18_1984,119050.95
1932 Model A Ford J-Coupe,S18_2325,Vintage Cars,S18_2325,109992.01
1965 Aston Martin DB5,S18_1589,Classic Cars,S18_1589,101778.13
1999 Indy 500 Monte Carlo SS,S18_2870,Classic Cars,S18_2870,100770.12
1968 Dodge Charger,S12_3380,Classic Cars,S12_3380,98718.76
America West Airlines B757-200,S700_2466,Planes,S700_2466,89347.8
2002 Chevy Corvette,S24_3432,Classic Cars,S24_3432,87404.81
1982 Ducati 996 R,S32_2206,Motorcycles,S32_2206,33268.76


Classic Cars and Motorcycles dominate the list of top-performing products, indicating strong customer preference for these categories

## most profitable customers

In [22]:
%%sql
WITH customer_orders AS (
  SELECT orders.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS profit
  FROM products
  JOIN orderdetails
  ON products.productCode = orderdetails.productCode
  JOIN orders
  ON orders.orderNumber = orderdetails.orderNumber
  GROUP BY orders.customerNumber
)

SELECT c.customerName, c.customerNumber, c.contactFirstName, c.contactLastName, c.country, co.profit
FROM customers c
JOIN customer_orders co
ON c.customerNumber = co.customerNumber
ORDER BY co.profit DESC
LIMIT 5;

 * sqlite:///stores.db
Done.


customerName,customerNumber,contactFirstName,contactLastName,country,profit
Euro+ Shopping Channel,141,Diego,Freyre,Spain,326519.66
Mini Gifts Distributors Ltd.,124,Susan,Nelson,USA,236769.39
Muscle Machine Inc,151,Jeff,Young,USA,72370.09
"Australian Collectors, Co.",114,Peter,Ferguson,Australia,70311.07
La Rochelle Gifts,119,Janine,Labrune,France,60875.3


The most profitable customers, such as customer number 141 (Euro+ Shopping Channel, Spain) and 124 (Mini Gifts Distributors Ltd., USA), generate profits exceeding $300,000 and $230,000 respectively. The top 5 customers contribute significantly to overall profitability, with a notable concentration in the USA and Europe.

## least profitable customers

In [20]:
%%sql
WITH customer_orders AS (
  SELECT orders.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS profit
  FROM products
  JOIN orderdetails
  ON products.productCode = orderdetails.productCode
  JOIN orders
  ON orders.orderNumber = orderdetails.orderNumber
  GROUP BY orders.customerNumber
)

SELECT c.customerName, c.customerNumber, c.contactFirstName, c.contactLastName, c.country, co.profit
FROM customers c
JOIN customer_orders co
ON c.customerNumber = co.customerNumber
ORDER BY co.profit
LIMIT 5;

 * sqlite:///stores.db
Done.


customerName,customerNumber,contactFirstName,contactLastName,country,profit
Boards & Toys Co.,219,Mary,Young,USA,2610.870000000001
Auto-Moto Classics Inc.,198,Leslie,Taylor,USA,6586.0199999999995
Frau da Collezione,473,Franco,Ricotti,Italy,9532.93
Atelier graphique,103,Carine,Schmitt,France,10063.800000000001
"Double Decker Gift Stores, Ltd",489,Thomas,Smith,UK,10868.04


The least profitable customers generate minimal revenue. It may be beneficial to analyze their purchasing patterns and consider targeted marketing or cross-selling strategies to increase their value, or to focus resources on more profitable segments.

## customer acquisition trends

In [9]:
%%sql

WITH

payment_with_year_month_table AS (
  SELECT *,
         CAST(SUBSTR(paymentDate, 1, 4) AS INTEGER)*100 + CAST(SUBSTR(paymentDate, 6, 7) AS INTEGER) AS year_month
    FROM payments p
),

customers_by_month_table AS (
  SELECT p1.year_month, COUNT(*) AS number_of_customers, SUM(p1.amount) AS total
    FROM payment_with_year_month_table p1
   GROUP BY p1.year_month
),

new_customers_by_month_table AS (
  SELECT p1.year_month,
         COUNT(DISTINCT customerNumber) AS number_of_new_customers,
         SUM(p1.amount) AS new_customer_total,
         (SELECT number_of_customers
            FROM customers_by_month_table c
          WHERE c.year_month = p1.year_month) AS number_of_customers,
         (SELECT total
            FROM customers_by_month_table c
           WHERE c.year_month = p1.year_month) AS total
    FROM payment_with_year_month_table p1
   WHERE p1.customerNumber NOT IN (SELECT customerNumber
                                     FROM payment_with_year_month_table p2
                                    WHERE p2.year_month < p1.year_month)
   GROUP BY p1.year_month
)

SELECT year_month,
       ROUND(number_of_new_customers * 100.0 / number_of_customers) AS number_of_new_customers_props,
       ROUND(new_customer_total * 100.0 / total) AS new_customers_total_props
  FROM new_customers_by_month_table;


 * sqlite:///stores.db
Done.


year_month,number_of_new_customers_props,new_customers_total_props
200301,100.0,100.0
200302,100.0,100.0
200303,100.0,100.0
200304,100.0,100.0
200305,83.0,100.0
200306,100.0,100.0
200307,75.0,68.0
200308,67.0,54.0
200309,80.0,96.0
200310,69.0,69.0


The proportion of new customers was very high at the beginning of the period, then gradually declined. This suggests initial market penetration followed by a plateau or market saturation. Revitalizing marketing efforts or exploring new markets may be necessary to boost new customer acquisition.

## LTV

In [10]:
%%sql

WITH

money_in_by_customer_table AS (
SELECT o.customerNumber, SUM(quantityOrdered * (priceEach - buyPrice)) AS revenue
  FROM products p
  JOIN orderdetails od
    ON p.productCode = od.productCode
  JOIN orders o
    ON o.orderNumber = od.orderNumber
 GROUP BY o.customerNumber
)

SELECT AVG(mc.revenue) AS ltv
  FROM money_in_by_customer_table mc;

 * sqlite:///stores.db
Done.


ltv
39039.5943877551


The average customer lifetime value is about $39,040, which is substantial. This underscores the importance of focusing on customer retention and satisfaction, as each customer represents significant long-term revenue potential.